In [1]:
"""
Name: movement.ipynb
Authors: Stephan Meighen-Berger, Martina Karl
Example how to create a video of the simulation
Requires mpl_toolkits and imageio
"""

'\nName: fd_video.ipynb\nAuthors: Stephan Meighen-Berger, Martina Karl\nExample how to create a video of the simulation\nRequires mpl_toolkits and imageio\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import matplotlib.colors as colors
import imageio

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day as FD
from fourth_day import config

In [6]:
config['geometry']['box size'] = 5e-1
config['geometry']['bounding box'] = 5.1e-1
config['organisms']['emission fraction'] = 0.5

In [7]:
# Creating a fourth_day object
fd = FD()

In [8]:
# Launching solver
fd.sim()

In [9]:
# Plotting standards
std_size = 10.
fontsize = 20.
lw=3.
h_length=1.5

In [10]:
# Animation stack
fontsize = 15.
# The frames are created in a loop
cmap = plt.cm.inferno
norm = colors.Normalize(vmin=0., vmax=1.)
for i, _ in tqdm(enumerate(fd.t)):
    figure = plt.figure(figsize=(std_size, std_size * 6. / 8.))
    ax = figure.add_subplot(111)
    ax.scatter(fd.statistics[i].loc[:, 'pos_x'].values,
               fd.statistics[i].loc[:, 'pos_y'].values,
               color=cmap(norm(fd.statistics[i].loc[:, 'energy'].values)),
               s=50.
    )
    ax.title=ax.set_title('Population t = %.f s' %fd.t[i], fontsize=fontsize)
    ax.set_xlim(-fd._world.bounding_box / 2., fd._world.bounding_box / 2.)
    ax.set_ylim(-fd._world.bounding_box / 2., fd._world.bounding_box / 2.)
    ax.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    figure.savefig(PICS + '\\Frames\\frame_%d.png' %i)
    plt.close(figure)

100it [00:08, 11.27it/s]


In [11]:
images = []
filenames = np.array([
    PICS + 'Frames\\frame_%d.png' %i
    for i in range(len(fd.t))
])
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(PICS + '%s.gif' %config['geometry']['volume'],
                images, duration=0.1)